In [6]:
!pip install zeugma
!pip install deslib

In [7]:
from zeugma.embeddings import EmbeddingTransformer

In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [9]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from preprocessing import *
from function import *

In [11]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from deslib.util.aggregation import *

In [12]:
w2v = EmbeddingTransformer('word2vec')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [13]:
glove = EmbeddingTransformer('glove')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [14]:
fasttext = EmbeddingTransformer('fasttext')

[==================================================] 100.0% 958.5/958.4MB downloaded


In [15]:
fold = "F1"

In [16]:
uri_train = 'https://raw.githubusercontent.com/wvs2/data-hate/master/Folds/{}_train_union.csv'.format(fold)
uri_val   = 'https://raw.githubusercontent.com/wvs2/data-hate/master/Folds/{}_train_union.csv'.format(fold)
uri_teste = 'https://raw.githubusercontent.com/wvs2/data-hate/master/Folds/{}_test_union.csv'.format(fold)

df_train = pd.read_table(uri_train, sep=',')
df_val   = pd.read_table(uri_val, sep=',')
df_test = pd.read_table(uri_teste, sep=',')

In [17]:
train = df_train['text'].fillna(' ').apply(pre_processing)
val = df_val['text'].fillna(' ').apply(pre_processing)
test, class_test = df_test['text'].fillna(' ').apply(pre_processing), df_test['class']

In [18]:
cv = CountVectorizer(analyzer='word', lowercase=True, stop_words='english')
cv.fit_transform(train.values.astype('U'))
     
tfidf =  TfidfVectorizer(analyzer='word', lowercase=True, use_idf=True, stop_words='english')
tfidf.fit_transform(train.values.astype('U'))

<24104x20172 sparse matrix of type '<class 'numpy.float64'>'
	with 164720 stored elements in Compressed Sparse Row format>

In [19]:
svm = {
    'CV': {
        'CLF': SVC(random_state=42, verbose=100, kernel='linear', gamma=0.1, probability=True),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': SVC(random_state=42, verbose=100, kernel='linear', gamma=0.1, probability=True),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': SVC(random_state=42, verbose=100, kernel='rbf', gamma=1, probability=True),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': SVC(random_state=42, verbose=100, kernel='rbf', gamma=0.5, probability=True),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': SVC(random_state=42, verbose=100, kernel='rbf', gamma=1, probability=True),
        'EXT': fasttext,
    }
}

In [20]:
svm_clfs = {

}
for ext, clf in svm.items():
    svm_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [21]:
df_pred_train = pd.DataFrame(df_train['class'])
df_pred_val = pd.DataFrame(df_val['class'])
df_pred_test = pd.DataFrame(df_test['class'])

df_prob_val = pd.DataFrame(df_val['class'])
df_prob_train = pd.DataFrame(df_train['class'])
df_prob_test = pd.DataFrame(df_test['class'])
for ext, clf in svm_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["SVM-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["SVM-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["SVM-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "SVM-{}-{}".format(ext,clf.classes_[0]), 
    "SVM-{}-{}".format(ext,clf.classes_[1]), 
    "SVM-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [22]:
lr = {
    'CV': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': fasttext,
    }
}

In [23]:
lr_clfs = {

}
for ext, clf in lr.items():
    lr_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

In [24]:
for ext, clf in lr_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["LR-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["LR-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["LR-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "LR-{}-{}".format(ext,clf.classes_[0]), 
    "LR-{}-{}".format(ext,clf.classes_[1]), 
    "LR-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [25]:
rf = {
    'CV': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': fasttext,
    }
}

In [26]:
rf_clfs = {

}
for ext, clf in rf.items():
    rf_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
building tree 1 of 50
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
building tree 2 of 50
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s
building tree 3 of 50
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.1s remaining:    0.0s
building tree 4 of 50
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.5s remaining:    0.0s
building tree 5 of 50
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.9s remaining:    0.0s
building tree 6 of 50
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    2.2s remaining:    0.0s
building tree 7 of 50
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    2.6s remaining:    0.0s
building tree 8 of 50
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    3.0s remaining:    0.0s
building tree 9 of 50
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    3.3s remaining:    0.0s
b

In [27]:
for ext, clf in rf_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["RF-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["RF-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["RF-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "RF-{}-{}".format(ext,clf.classes_[0]), 
    "RF-{}-{}".format(ext,clf.classes_[1]), 
    "RF-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

In [28]:
nb = {
    'CV': {
        'CLF': MultinomialNB(alpha=0.5, fit_prior=False),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': MultinomialNB(alpha=0.5, fit_prior=False),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': BernoulliNB(alpha=0.5, fit_prior=True),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': BernoulliNB(alpha=0.1, fit_prior=True),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': BernoulliNB(alpha=0.1, fit_prior=True),
        'EXT': fasttext,
    }
}

In [29]:
nb_clfs = {

}
for ext, clf in nb.items():
    nb_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

In [30]:
for ext, clf in nb_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["NB-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["NB-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["NB-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "NB-{}-{}".format(ext,clf.classes_[0]), 
    "NB-{}-{}".format(ext,clf.classes_[1]), 
    "NB-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [31]:
mlp = {
    'CV': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='lbfgs'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT': fasttext,
    }
}

In [32]:
mlp_clfs = {

}
for ext, clf in mlp.items():
    mlp_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Iteration 1, loss = 0.43566899
Iteration 2, loss = 0.17838093
Iteration 3, loss = 0.10860688
Iteration 4, loss = 0.07645685
Iteration 5, loss = 0.05924640
Iteration 6, loss = 0.04913880
Iteration 7, loss = 0.04266982
Iteration 8, loss = 0.03931920
Iteration 9, loss = 0.03602393
Iteration 10, loss = 0.03377310
Iteration 11, loss = 0.03201866
Iteration 12, loss = 0.03095476
Iteration 13, loss = 0.02942311
Iteration 14, loss = 0.02785375
Iteration 15, loss = 0.02724985
Iteration 16, loss = 0.02768992
Iteration 17, loss = 0.02677937
Iteration 18, loss = 0.02635244
Iteration 19, loss = 0.02572574
Iteration 20, loss = 0.02475292


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.54956900
Iteration 2, loss = 0.47358450
Iteration 3, loss = 0.45502435
Iteration 4, loss = 0.44129354
Iteration 5, loss = 0.42782614
Iteration 6, loss = 0.41589613
Iteration 7, loss = 0.40334218
Iteration 8, loss = 0.39432378
Iteration 9, loss = 0.38177470
Iteration 10, loss = 0.37005422
Iteration 11, loss = 0.36004882
Iteration 12, loss = 0.35012824
Iteration 13, loss = 0.33991732
Iteration 14, loss = 0.33090417
Iteration 15, loss = 0.31935060
Iteration 16, loss = 0.31085641
Iteration 17, loss = 0.30214171
Iteration 18, loss = 0.29233245
Iteration 19, loss = 0.28522540
Iteration 20, loss = 0.27486916


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.62633808
Iteration 2, loss = 0.58693991
Iteration 3, loss = 0.57643806
Iteration 4, loss = 0.57076906
Iteration 5, loss = 0.56549857
Iteration 6, loss = 0.56105752
Iteration 7, loss = 0.55847894
Iteration 8, loss = 0.55507253
Iteration 9, loss = 0.55242536
Iteration 10, loss = 0.54933686
Iteration 11, loss = 0.54790295
Iteration 12, loss = 0.54548187
Iteration 13, loss = 0.54360937
Iteration 14, loss = 0.54220443
Iteration 15, loss = 0.54041833
Iteration 16, loss = 0.53815266
Iteration 17, loss = 0.53770827
Iteration 18, loss = 0.53499559
Iteration 19, loss = 0.53377311
Iteration 20, loss = 0.53269102


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.58535360
Iteration 2, loss = 0.47033985
Iteration 3, loss = 0.44894031
Iteration 4, loss = 0.43560922
Iteration 5, loss = 0.42664042
Iteration 6, loss = 0.41754205
Iteration 7, loss = 0.41077510
Iteration 8, loss = 0.40529489
Iteration 9, loss = 0.39952566
Iteration 10, loss = 0.39399540
Iteration 11, loss = 0.38948081
Iteration 12, loss = 0.38563632
Iteration 13, loss = 0.37956074
Iteration 14, loss = 0.37551100
Iteration 15, loss = 0.36838989
Iteration 16, loss = 0.36398400
Iteration 17, loss = 0.35957169
Iteration 18, loss = 0.35470471
Iteration 19, loss = 0.35222254
Iteration 20, loss = 0.34607371


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [33]:
for ext, clf in mlp_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["MLP-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["MLP-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["MLP-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "MLP-{}-{}".format(ext,clf.classes_[0]), 
    "MLP-{}-{}".format(ext,clf.classes_[1]), 
    "MLP-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [34]:
extra = {
    'CV': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=20),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': fasttext,
    }
}

In [35]:
extra_clfs = {

}
for ext, clf in extra.items():
    extra_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
building tree 1 of 50
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
building tree 2 of 50
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s
building tree 3 of 50
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.8s remaining:    0.0s
building tree 4 of 50
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.3s remaining:    0.0s
building tree 5 of 50
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.9s remaining:    0.0s
building tree 6 of 50
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    3.5s remaining:    0.0s
building tree 7 of 50
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    4.1s remaining:    0.0s
building tree 8 of 50
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    4.7s remaining:    0.0s
building tree 9 of 50
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    5.3s remaining:    0.0s
b

In [36]:
for ext, clf in extra_clfs.items():
  # Predic  t
  df_train_ = pd.DataFrame(clf.predict(train), columns=["EXTRA-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["EXTRA-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["EXTRA-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "EXTRA-{}-{}".format(ext,clf.classes_[0]), 
    "EXTRA-{}-{}".format(ext,clf.classes_[1]), 
    "EXTRA-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

In [37]:
knn = {
    'CV': {
        'CLF': KNeighborsClassifier(n_neighbors=3, algorithm='auto'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': KNeighborsClassifier(n_neighbors=3, algorithm='auto'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='ball_tree'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT': fasttext,
    }
}

In [38]:
knn_clfs = { }
for ext, clf in knn.items():
    knn_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

In [39]:
for ext, clf in knn_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["KNN-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["KNN-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["KNN-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "KNN-{}-{}".format(ext,clf.classes_[0]), 
    "KNN-{}-{}".format(ext,clf.classes_[1]), 
    "KNN-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


# CNN

In [40]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH=300

y_train = to_categorical(df_train['class'])
y_val = to_categorical(df_val['class'])

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train)

seq_train = tokenizer.texts_to_sequences(train)
seq_val = tokenizer.texts_to_sequences(val)
seq_test = tokenizer.texts_to_sequences(test)

data_train = pad_sequences(seq_train, maxlen=MAX_SEQUENCE_LENGTH)
data_val = pad_sequences(seq_val, maxlen=MAX_SEQUENCE_LENGTH)
data_test = pad_sequences(seq_test, maxlen=MAX_SEQUENCE_LENGTH)

In [73]:
def get_CNN(ext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, MAX_SEQUENCE_LENGTH=300, activation='sigmoid', word_embedding=False):
    if word_embedding==False:
        X_ext = ext.get_feature_names()
        model = Word2Vec([X_ext], min_count=1, workers=1, size=300)
    else:
        model = ext.model

    word_index = tokenizer.word_index
    
    nb_words = min(MAX_NB_WORDS, len(word_index))+1
    embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
    for word, i in word_index.items():
      if i >= MAX_NB_WORDS:
        continue
      if word in model.wv.vocab:
        embedding_matrix[i] = model.wv.word_vec(word)

    embedding_layer = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False
    )
    cnn = Sequential()
    cnn.add(embedding_layer)
    cnn.add(Dropout(0.2))
    cnn.add(Conv1D(64, 3, padding='valid', activation='relu', strides=1))
    cnn.add(GlobalMaxPooling1D())
    cnn.add(Dense(256))
    cnn.add(Dropout(0.2))
    cnn.add(Activation('relu'))
    cnn.add(Dense(3))
    cnn.add(Activation(activation))
    cnn.summary()
    cnn.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return cnn

In [74]:
cnn_cv = get_CNN(cv, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid')
cnn_tfidf = get_CNN(tfidf, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='softmax')
cnn_w2v = get_CNN(w2v, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True)
cnn_glove = get_CNN(glove, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=25, activation='sigmoid', word_embedding=True)
cnn_fast = get_CNN(fasttext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 300, 300)          6000300   
_________________________________________________________________
dropout_4 (Dropout)          (None, 300, 300)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 298, 64)           57664     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               16640     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_4 (Activation)    (None, 256)              

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  app.launch_new_instance()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 300, 300)          6000300   
_________________________________________________________________
dropout_8 (Dropout)          (None, 300, 300)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 298, 64)           57664     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               16640     
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_8 (Activation)    (None, 256)              

In [75]:
cnn_cv.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=20)

Epoch 1/20
1206/1206 [==============================] - 102s 84ms/step - loss: 0.6393 - accuracy: 0.7135 - val_loss: 0.5340 - val_accuracy: 0.7667
Epoch 2/20
1206/1206 [==============================] - 101s 84ms/step - loss: 0.4872 - accuracy: 0.7778 - val_loss: 0.4375 - val_accuracy: 0.8196
Epoch 3/20
1206/1206 [==============================] - 102s 84ms/step - loss: 0.4182 - accuracy: 0.8289 - val_loss: 0.3805 - val_accuracy: 0.8496
Epoch 4/20
1206/1206 [==============================] - 99s 82ms/step - loss: 0.3835 - accuracy: 0.8464 - val_loss: 0.3644 - val_accuracy: 0.8579
Epoch 5/20
1206/1206 [==============================] - 99s 82ms/step - loss: 0.3561 - accuracy: 0.8579 - val_loss: 0.3643 - val_accuracy: 0.8587
Epoch 6/20
1206/1206 [==============================] - 99s 82ms/step - loss: 0.3361 - accuracy: 0.8699 - val_loss: 0.3081 - val_accuracy: 0.8799
Epoch 7/20
1206/1206 [==============================] - 99s 82ms/step - loss: 0.3228 - accuracy: 0.8734 - val_loss: 0.302

In [76]:
cnn_tfidf.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_w2v.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_glove.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_fast.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)

Epoch 1/20
121/121 [==============================] - 92s 758ms/step - loss: 0.8465 - accuracy: 0.6351 - val_loss: 0.7146 - val_accuracy: 0.6379
Epoch 2/20
121/121 [==============================] - 91s 756ms/step - loss: 0.6267 - accuracy: 0.7081 - val_loss: 0.5684 - val_accuracy: 0.7525
Epoch 3/20
121/121 [==============================] - 92s 758ms/step - loss: 0.5543 - accuracy: 0.7511 - val_loss: 0.5255 - val_accuracy: 0.7612
Epoch 4/20
121/121 [==============================] - 92s 758ms/step - loss: 0.5190 - accuracy: 0.7638 - val_loss: 0.4892 - val_accuracy: 0.7741
Epoch 5/20
121/121 [==============================] - 91s 757ms/step - loss: 0.4841 - accuracy: 0.7747 - val_loss: 0.4526 - val_accuracy: 0.7867
Epoch 6/20
121/121 [==============================] - 91s 756ms/step - loss: 0.4517 - accuracy: 0.8015 - val_loss: 0.4295 - val_accuracy: 0.8308
Epoch 7/20
121/121 [==============================] - 91s 756ms/step - loss: 0.4255 - accuracy: 0.8272 - val_loss: 0.4079 - val_ac

In [77]:
# print(df_prob_train['CNN-CV-0'])
# # for ext in ['CV', 'TFIDF', 'W2V', 'GLOVE', 'FAST']:
# #   for i in range(0,3):
# #     df_prob_train = df_prob_train.drop(columns=['CNN-{}-{}'.format(ext, i)])
# #     df_prob_val = df_prob_val.drop(columns=['CNN-{}-{}'.format(ext, i)])
# #     df_prob_test = df_prob_test.drop(columns=['CNN-{}-{}'.format(ext, i)])

In [78]:
print(df_prob_train.columns)

Index(['class', 'SVM-CV-0', 'SVM-CV-1', 'SVM-CV-2', 'SVM-TFIDF-0',
       'SVM-TFIDF-1', 'SVM-TFIDF-2', 'SVM-W2V-0', 'SVM-W2V-1', 'SVM-W2V-2',
       ...
       'KNN-TFIDF-2', 'KNN-W2V-0', 'KNN-W2V-1', 'KNN-W2V-2', 'KNN-GLOVE-0',
       'KNN-GLOVE-1', 'KNN-GLOVE-2', 'KNN-FAST-0', 'KNN-FAST-1', 'KNN-FAST-2'],
      dtype='object', length=106)


In [79]:
# df_pred_test = df_pred_test.drop(['CNN-CV'], axis=1)
# print(df_pred_test.columns)
# CV
cols = ["CNN-CV-0", "CNN-CV-1", "CNN-CV-2"]
df_train_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_train), axis=1), columns=["CNN-CV"])
df_val_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_val), axis=1), columns=["CNN-CV"])
df_test_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_test), axis=1), columns=["CNN-CV"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# print(df_pred_test.columns)

# # Probabilidades
  
df_train_ = pd.DataFrame(cnn_cv.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_cv.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_cv.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


cols = ["CNN-TF-0", "CNN-TF-1", "CNN-TF-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_train), axis=1), columns=["CNN-TFIDF"])
df_val_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_val), axis=1), columns=["CNN-TFIDF"])
df_test_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_test), axis=1), columns=["CNN-TFIDF"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_tfidf.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_tfidf.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_tfidf.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

cols = ["CNN-W2V-0", "CNN-W2V-1", "CNN-W2V-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_train), axis=1), columns=["CNN-W2V"])
df_val_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_val), axis=1), columns=["CNN-W2V"])
df_test_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_test), axis=1), columns=["CNN-W2V"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_w2v.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_w2v.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_w2v.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


cols = ["CNN-GLOVE-0", "CNN-GLOVE-1", "CNN-GLOVE-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_train), axis=1), columns=["CNN-GLOVE"])
df_val_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_val), axis=1), columns=["CNN-GLOVE"])
df_test_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_test), axis=1), columns=["CNN-GLOVE"])

df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_glove.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_glove.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_glove.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

cols = ["CNN-FAST-0", "CNN-FAST-1", "CNN-FAST-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_train), axis=1), columns=["CNN-FAST"])
df_val_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_val), axis=1), columns=["CNN-FAST"])
df_test_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_test), axis=1), columns=["CNN-FAST"])

df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_fast.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_fast.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_fast.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [80]:
df_prob_train['class']

0        1
1        1
2        2
3        2
4        2
        ..
24099    1
24100    2
24101    1
24102    1
24103    2
Name: class, Length: 24104, dtype: int64

In [81]:
df_pred_train.to_csv("pred_train_union_{}.csv".format(fold))
df_pred_val.to_csv("pred_val_union_{}.csv".format(fold))
df_pred_test.to_csv("pred_test_union_{}.csv".format(fold))

In [82]:
df_prob_train.to_csv("prob_train_union_{}.csv".format(fold))
df_prob_val.to_csv("prob_val_union_{}.csv".format(fold))
df_prob_test.to_csv("prob_test_union{}.csv".format(fold))